# Scanners

In [1]:
from ib_insync import *
util.startLoop()  

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=17)

<IB connected to 127.0.0.1:7497 clientId=17>

## Basic Scanner

To create a scanner, create a `ScannerSubscription` to submit to the `reqScannerData` method. For any scanner to work, at least these three fields must be filled: `instrument` (the what), `locationCode` (the where), and `scanCode` (the ranking).

For example, to find the top ranked US stock percentage gainers:

In [2]:
sub = ScannerSubscription(
    instrument='STK', 
    locationCode='STK.US.MAJOR', 
    scanCode='TOP_PERC_GAIN')

scanData = ib.reqScannerData(sub)

print(f'{len(scanData)} results, first one:')
print(scanData[0])

50 results, first one:
ScanData(rank=0, contractDetails=ContractDetails(contract=Contract(secType='STK', conId=409806996, symbol='CTIB', exchange='SMART', currency='USD', localSymbol='CTIB', tradingClass='SCM'), marketName='SCM', minTick=0.0, orderTypes='', validExchanges='', priceMagnifier=0, underConId=0, longName='', contractMonth='', industry='', category='', subcategory='', timeZoneId='', tradingHours='', liquidHours='', evRule='', evMultiplier=0, mdSizeMultiplier=0, aggGroup=0, underSymbol='', underSecType='', marketRuleIds='', secIdList=[], realExpirationDate='', lastTradeTime='', stockType='', cusip='', ratings='', descAppend='', bondType='', couponType='', callable=False, putable=False, coupon=0, convertible=False, maturity='', issueDate='', nextOptionDate='', nextOptionType='', nextOptionPartial=False, notes=''), distance='', benchmark='', projection='', legsStr='')


*The displayed error 162 can be ignored*

The scanner returns a list of contract details, without current market data (this can be obtained via seperate market data requests).

## Filtering scanner results, the old way

The `ScannerSubscription` object has addional parameters that can be set to filter the results, such as `abovePrice`, `aboveVolume`, `marketCapBelow` or `spRatingAbove`. 

For example, to reuse the previous `sub` and query only for stocks with a price above 200 dollar:

In [3]:
sub.abovePrice = 200
scanData = ib.reqScannerData(sub)

symbols = [sd.contractDetails.contract.symbol for sd in scanData]
print(symbols)

Error 162, reqId 3: Historical Market Data Service error message:API scanner subscription cancelled: 3


['FVRR', 'BA', 'AMGN', 'VRTX', 'LMT']


## Filtering, the new way

In the new way there is a truly vast number of parameters available to use for filtering.
These new scanner parameters map directly to the options available through the TWS "Advanced Market Scanner." The parameters
are dynamically available from a huge XML document that is returned by ``reqScannerParameters``:

In [4]:
xml = ib.reqScannerParameters()

print(len(xml), 'bytes')

Error 162, reqId 4: Historical Market Data Service error message:API scanner subscription cancelled: 4


2000323 bytes


To view the XML in a web browser:

In [5]:
path = 'scanner_parameters.xml'
with open(path, 'w') as f:
    f.write(xml)

import webbrowser
webbrowser.open(path)

True

In [6]:
# parse XML document
import xml.etree.ElementTree as ET
tree = ET.fromstring(xml)

# find all tags that are available for filtering
tags = [elem.text for elem in tree.findall('.//AbstractField/code')]
print(len(tags), 'tags:')
print(tags)

1352 tags:
['priceAbove', 'priceBelow', 'usdPriceAbove', 'usdPriceBelow', 'volumeAbove', 'usdVolumeAbove', 'usdVolumeBelow', 'avgVolumeAbove', 'avgVolumeBelow', 'avgUsdVolumeAbove', 'avgUsdVolumeBelow', 'ihNumSharesInsiderAbove', 'ihNumSharesInsiderBelow', 'ihInsiderOfFloatPercAbove', 'ihInsiderOfFloatPercBelow', 'iiNumSharesInstitutionalAbove', 'iiNumSharesInstitutionalBelow', 'iiInstitutionalOfFloatPercAbove', 'iiInstitutionalOfFloatPercBelow', 'marketCapAbove1e6', 'marketCapBelow1e6', 'moodyRatingAbove', 'moodyRatingBelow', 'spRatingAbove', 'spRatingBelow', 'ratingsRelation', 'bondCreditRating', 'maturityDateAbove', 'maturityDateBelow', 'currencyLike', 'excludeConvertible', 'couponRateAbove', 'couponRateBelow', 'optVolumeAbove', 'optVolumeBelow', 'avgOptVolumeAbove', 'optVolumePCRatioAbove', 'optVolumePCRatioBelow', 'impVolatAbove', 'impVolatBelow', 'impVolatOverHistAbove', 'impVolatOverHistBelow', 'imbalanceAbove', 'imbalanceBelow', 'displayImbalanceAdvRatioAbove', 'displayImbalanc

Notice how ``abovePrice`` is now called ``priceAbove``...

Using three of these filter tags, let's perform a query to find all US stocks that went up 20% and have a current price between 5 and 50 dollar, sorted by percentage gain:

In [7]:
sub = ScannerSubscription(
    instrument='STK',
    locationCode='STK.US.MAJOR',
    scanCode='TOP_PERC_GAIN')

tagValues = [
    TagValue("changePercAbove", "20"),
    TagValue('priceAbove', 5),
    TagValue('priceBelow', 50)]

# the tagValues are given as 3rd argument; the 2nd argument must always be an empty list
# (IB has not documented the 2nd argument and it's not clear what it does)
scanData = ib.reqScannerData(sub, [], tagValues)

symbols = [sd.contractDetails.contract.symbol for sd in scanData]
print(symbols)

[]


Any scanner query that TWS can do can alse be done through the API. The `scanCode` parameter maps directly to the "Parameter" window in the TWS "Advanced Market Scanner." We can verify this by printing out the `scanCode` values available:

In [8]:
scanCodes = [e.text for e in tree.findall('.//scanCode')]

print(len(scanCodes), 'scan codes, showing the ones starting with "TOP":')
print([sc for sc in scanCodes if sc.startswith('TOP')])

762 scan codes, showing the ones starting with "TOP":
['TOP_PERC_GAIN', 'TOP_PERC_LOSE', 'TOP_TRADE_COUNT', 'TOP_TRADE_RATE', 'TOP_PRICE_RANGE', 'TOP_VOLUME_RATE', 'TOP_OPEN_PERC_GAIN', 'TOP_OPEN_PERC_LOSE', 'TOP_AFTER_HOURS_PERC_GAIN', 'TOP_AFTER_HOURS_PERC_LOSE', 'TOP_OPT_IMP_VOLAT_GAIN', 'TOP_OPT_IMP_VOLAT_LOSE', 'TOP_STOCK_BUY_IMBALANCE_ADV_RATIO', 'TOP_STOCK_SELL_IMBALANCE_ADV_RATIO', 'TOP_STOCK_BUY_REG_IMBALANCE_ADV_RATIO', 'TOP_STOCK_SELL_REG_IMBALANCE_ADV_RATIO', 'TOP_PERC_GAIN', 'TOP_PERC_LOSE', 'TOP_TRADE_RATE', 'TOP_PERC_GAIN']


Queries are not limited to stocks. To get a list of all supported instruments:

In [9]:
instrumentTypes = set(e.text for e in tree.findall('.//Instrument/type'))
print(instrumentTypes)

{'BOND.AGNCY', 'ETF.FI.US', 'STOCK.EU', 'SSF.NA', 'BOND.MUNI', 'BOND.GOVT', 'BOND.CD', 'SSF.HK', 'WAR.EU', 'STOCK.HK', 'SSF.EU', 'IND.EU', 'IND.HK', 'FUND.ALL', 'STK', 'FUT.HK', 'BOND', 'FUT.US', 'STOCK.NA', 'BOND.GOVT.NON-US', 'PORTSTK', 'Global', 'IND.US', 'FUT.EU', 'STOCK.ME', 'SLB.US', 'NATCOMB', 'FUT.NA', 'ETF.EQ.US'}


To find all location codes:

In [10]:
locationCodes = [e.text for e in tree.findall('.//locationCode')]
print(locationCodes)

['STK.US', 'STK.US.MAJOR', 'STK.NYSE', 'STK.AMEX', 'STK.ARCA', 'STK.NASDAQ', 'STK.NASDAQ.NMS', 'STK.NASDAQ.SCM', 'STK.BATS', 'STK.US.MINOR', 'STK.PINK', 'STK.OTCBB', 'ETF.EQ.US', 'ETF.EQ.US.MAJOR', 'ETF.EQ.ARCA', 'ETF.EQ.NASDAQ.NMS', 'ETF.EQ.BATS', 'ETF.FI.US', 'ETF.FI.US.MAJOR', 'ETF.FI.ARCA', 'ETF.FI.NASDAQ.NMS', 'ETF.FI.BATS', 'FUT.US', 'FUT.GLOBEX', 'FUT.ECBOT', 'FUT.IPE', 'FUT.NYBOT', 'FUT.NYMEX', 'FUT.NYSELIFFE', 'FUT.CFE', 'FUT.CFECRYPTO', 'FUT.CMECRYPTO', 'FUT.ICECRYPTO', 'IND.US', 'BOND.WW', 'BOND.US', 'BOND.EU.EURONEXT', 'BOND.CD.US', 'BOND.AGNCY.US', 'BOND.GOVT.US', 'BOND.MUNI.US', 'BOND.GOVT.NON-US', 'BOND.GOVT.US.NON-US', 'BOND.GOVT.EU.EURONEXT', 'BOND.GOVT.HK.SEHK', 'SLB.PREBORROW', 'FUND.ALL', 'FUND.US', 'FUND.NON-US', 'STK.NA', 'STK.NA.CANADA', 'STK.NA.TSE', 'STK.NA.VENTURE', 'STK.NA.MEXI', 'FUT.NA', 'FUT.NA.CDE', 'FUT.NA.MEXDER', 'SSF.NA', 'SSF.NA.MEXDER', 'STK.EU', 'STK.EU.VSE', 'STK.EU.SBF', 'STK.EU.IBIS', 'STK.EU.IBIS-XETRA', 'STK.EU.IBIS-EUSTARS', 'STK.EU.IBIS-USST

In [11]:
ib.disconnect()